In [1]:
import pandas as pd
from pathlib import Path
import json
import numpy as np
from tqdm import tqdm

from scipy.stats import ttest_rel

In [2]:
sentiments_dir = Path('sentiments')

df_emoji = pd.read_excel(sentiments_dir / 'emetophobia_emojis_statistics.xlsx')
df_no_emoji = pd.read_excel(sentiments_dir / 'emetophobia_noemojis_statistics.xlsx')

In [3]:
df_emoji.head()

,emoji,min,max,mean,std,nums
0,😭,-0.9966,0.9859,-0.672362,0.577602,91
1,😣,-0.9810,-0.6240,-0.757600,0.140943,6
2,😔,-0.9178,0.0360,-0.619875,0.344388,8
3,😳,-0.8317,-0.8317,-0.831700,0.000000,2
4,🥹,-0.9917,-0.9433,-0.975567,0.022816,3


In [4]:
with open(sentiments_dir / 'emetophobia_posts_emoji_sentiment.json', 'r') as f:
    emoji_dict = json.load(f)
with open(sentiments_dir / 'emetophobia_posts_noemoji_sentiment.json', 'r') as f:
    noemoji_dict = json.load(f)

In [5]:
df_diff = df_emoji.merge(df_no_emoji, on='emoji', how='inner', suffixes=('_with_emoji', '_without_emoji'))

df_diff['mean_difference'] = df_diff['mean_with_emoji'] - df_diff['mean_without_emoji']

# t_stat, p_value = ttest_rel(df_diff['mean_with_emoji'], df_diff['mean_without_emoji'])
for i in tqdm(range(len(df_diff))):
    emoji = df_diff.iloc[i]['emoji']
    if emoji in emoji_dict and emoji in noemoji_dict:
        t_stat, p_value = ttest_rel(emoji_dict[emoji], noemoji_dict[emoji])
        df_diff.at[i, 't_stat'] = t_stat
        df_diff.at[i, 'p_value'] = p_value
    else:
        df_diff.at[i, 't_stat'] = np.nan
        df_diff.at[i, 'p_value'] = np.nan
df_diff = df_diff.sort_values(by='nums_with_emoji', ascending=False)


  0%|          | 0/82 [00:00<?, ?it/s]

/home/tux/Desktop/Master/An1/Sem2/BioNLP/Proiect/venv/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
100%|██████████| 82/82 [00:00<00:00, 622.95it/s]


In [6]:
df_diff.head()

,emoji,min_with_emoji,max_with_emoji,mean_with_emoji,std_with_emoji,nums_with_emoji,min_without_emoji,max_without_emoji,mean_without_emoji,std_without_emoji,nums_without_emoji,mean_difference,t_stat,p_value
0,😭,-0.9966,0.9859,-0.672362,0.577602,91,-0.9954,0.9876,-0.444981,0.664632,91,-0.227380,-6.269718,1.228922e-08
9,🫶,-0.9885,0.9413,0.672383,0.536214,12,-0.9851,0.9505,0.599492,0.614665,12,0.072892,1.206261,2.530206e-01
25,❤,-0.9965,0.8932,-0.212490,0.780557,10,-0.9956,0.8932,-0.212260,0.780327,10,-0.000230,-1.943084,8.389310e-02
14,🥲,-0.9838,-0.7619,-0.883778,0.070573,9,-0.9867,-0.7619,-0.884100,0.071034,9,0.000322,1.000000,3.465935e-01
36,💔,-0.9195,0.0377,-0.607400,0.395321,8,-0.9456,-0.4659,-0.725550,0.188329,8,0.118150,1.238247,2.555378e-01


In [7]:
df_diff.to_excel(sentiments_dir / 'emetophobia_diff_statistics.xlsx', index=False)

In [8]:
all_sents_emoji = []
for emoji in emoji_dict:
    all_sents_emoji.extend(emoji_dict[emoji])

all_sents_noemoji = []
for emoji in noemoji_dict:
    all_sents_noemoji.extend(noemoji_dict[emoji])

t_stat, p_value = ttest_rel(all_sents_emoji, all_sents_noemoji)

t_stat, p_value

(-4.255487935548932, 2.660058870348046e-05)

In [9]:
with open(sentiments_dir / 'emetophobia_posts_sent_emoji.json', 'r') as f:
    posts_emoji = json.load(f)

with open(sentiments_dir / 'emetophobia_posts_sent_noemoji.json', 'r') as f:
    posts_noemoji = json.load(f)

posts_emoji[0], posts_noemoji[0]

({'title': 'Life-changing website for movie triggers!',
  'content': 'I just wanted to share a website/app that has been an absolute game changer for my emetophobia: doesthedogdie.com  It’s basically a big trigger warning database for movies where users flag content like animal death, violence, and, importantly for us, v*. You can check if a movie has v* scenes before watching, and the community often provides time stamps and context. I’ve been using it for a while and it has made such a difference. I can finally enjoy movies again without that constant fear in the back of my mind. I really encourage others here to check it out, and if you can, help others by contributing to the site when you come across triggering scenes. It’s a small thing that can help a lot of people.',
  'date': 1749890934.0,
  'ups': 2,
  'upvote_ratio': 1.0,
  'downvotes': 0,
  'labels': 'Techniques, tips and tricks',
  'sentiment': {'neg': 0.07, 'neu': 0.783, 'pos': 0.147, 'compound': 0.8563},
  'emojis': [],
 

In [10]:
all_sents_emoji = []
lbl_sents_emoji = {}

all_sents_noemoji = []
lbl_sents_noemoji = {}

for post in tqdm(posts_emoji):
    all_sents_emoji.append(post['sentiment']['compound'])

    if post['labels'] in lbl_sents_emoji:
        lbl_sents_emoji[post['labels']].append(post['sentiment']['compound'])
    else:
        lbl_sents_emoji[post['labels']] = [post['sentiment']['compound']]

for post in tqdm(posts_noemoji):
    all_sents_noemoji.append(post['sentiment']['compound'])

    if post['labels'] in lbl_sents_noemoji:
        lbl_sents_noemoji[post['labels']].append(post['sentiment']['compound'])
    else:
        lbl_sents_noemoji[post['labels']] = [post['sentiment']['compound']]

100%|██████████| 986/986 [00:00<00:00, 1717435.11it/s]


In [11]:
all_sents = {
    'max_with_emoji': np.max(all_sents_emoji),
    'min_with_emoji': np.min(all_sents_emoji),
    'mean_with_emoji': np.mean(all_sents_emoji),
    'std_with_emoji': np.std(all_sents_emoji),
    'max_without_emoji': np.max(all_sents_noemoji),
    'min_without_emoji': np.min(all_sents_noemoji),
    'mean_without_emoji': np.mean(all_sents_noemoji),
    'std_without_emoji': np.std(all_sents_noemoji),
}


t_stat_all, p_value_all = ttest_rel(all_sents_emoji, all_sents_noemoji)

all_sents['t_value'] = t_stat_all
all_sents['p_value'] = p_value_all

df_all_sents = pd.DataFrame.from_dict(all_sents, orient='index', columns=['value'])

t_stat_all, p_value_all

(-3.1540444865214092, 0.0016590158848331148)

In [12]:
df_all_sents.to_excel(sentiments_dir / 'emetophobia_all_sent_statistics.xlsx', index=True, index_label='metric')

In [13]:
lbl_sents = []
for lbl in lbl_sents_emoji:
    if len(lbl_sents_emoji[lbl]) <= 1:
        continue
    
    if lbl in lbl_sents_noemoji:
        t_stat, p_value = ttest_rel(lbl_sents_emoji[lbl], lbl_sents_noemoji[lbl])
    else:
        t_stat, p_value = np.nan, np.nan

    lbl_sents.append({
        'label': lbl,
        'mean_with_emoji': np.mean(lbl_sents_emoji[lbl]),
        'std_with_emoji': np.std(lbl_sents_emoji[lbl]),
        'mean_without_emoji': np.mean(lbl_sents_noemoji[lbl]),
        'std_without_emoji': np.std(lbl_sents_noemoji[lbl]),
        't_stat': t_stat,
        'p_value': p_value
    })

df_lbl_sents = pd.DataFrame.from_records(lbl_sents)

df_lbl_sents.to_excel(sentiments_dir / 'emetophobia_label_sent_statistics.xlsx', index=False)